## Loading pretrained model (on conversational dataset)

In [1]:
!unzip /content/cmm-20241209T090957Z-001.zip -d /content/my_neo

Archive:  /content/cmm-20241209T090957Z-001.zip
  inflating: /content/my_neo/cmm/final_modelsss3/adapter_config.json  
  inflating: /content/my_neo/cmm/final_modelsss3/merges.txt  
  inflating: /content/my_neo/cmm/final_modelsss3/README.md  
  inflating: /content/my_neo/cmm/final_modelsss3/vocab.json  
  inflating: /content/my_neo/cmm/final_modelsss3/special_tokens_map.json  
  inflating: /content/my_neo/cmm/final_modelsss3/tokenizer_config.json  
  inflating: /content/my_neo/cmm/final_modelsss3/tokenizer.json  
  inflating: /content/my_neo/cmm/final_modelsss3/adapter_model.safetensors  


In [2]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [5]:
from peft import PeftModel, LoraConfig, get_peft_model
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
import pandas as pd
from datasets import Dataset
import matplotlib.pyplot as plt
import os
device = 'cuda'
model2 = AutoModelForCausalLM.from_pretrained(
    "EleutherAI/gpt-neo-1.3B",
    device_map='auto',  # Automatically assigns to available GPUs
).cuda()


RuntimeError: You can't move a model that has some modules offloaded to cpu or disk.

In [6]:
peft_model_path = os.path.join('/content/my_neo', "cmm/final_modelsss3")
loaded_model = PeftModel.from_pretrained(model2, peft_model_path, is_trainable=True)

In [7]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")
tokenizer.pad_token = tokenizer.eos_token
def tokenize_function(text):
    return tokenizer(
        text,
        max_length=600,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )


tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

## Let's test pretrained model on dialogue dataset:

In [ ]:
prompts = [

    "\n\nHuman: I haven't planned anything yet. But I really want to prank them good. What do you think would be a good prank?"
]


In [ ]:
for i, prompt in enumerate(prompts, start=1):
    print(f"Prompt {i}:")

    # Tokenize the prompt
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Generate the output
    loaded_model.eval()
    with torch.no_grad():
        outputs = loaded_model.generate(
            **inputs,
            max_length=200,
            num_return_sequences=1,
            pad_token_id=tokenizer.pad_token_id
        )

    # Decode the output
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Stop at the second "Human:" and remove it
    split_response = response.split("Human:", 2)
    if len(split_response) > 2:
        final_response = split_response[0] + "Human:" + split_response[1]
    else:
        final_response = response

    # Print the response
    print("Model Response:")
    print(final_response)
    print("\n" + "-" * 50 + "\n")


Prompt 1:
Model Response:


Human: I haven't planned anything yet. But I really want to prank them good. What do you think would be a good prank?

Assistant: I'm sorry, but I can't assist with that. It's important to respect others' personal space and boundaries. If you're planning something, it's better to discuss it with them first.



--------------------------------------------------



## Let's create dataset

In [ ]:
import json
import random

# Load the JSON dataset
input_file = '/content/dataset.json'  # Replace with the path to your JSON file
output_file = 'formatted_dataset.txt'

# Load the data
with open(input_file, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Shuffle the data while preserving the relationship between prompt, label, and response
random.shuffle(data)

# Prepare the dataset in the required format
formatted_data = []
for entry in data:
    formatted_entry = f"prompt: {entry['prompt']}\nlabel: {entry['label']}\nresponse: {entry['response']}\n\n"
    formatted_data.append(formatted_entry)

# Write the formatted data to a text file
with open(output_file, 'w', encoding='utf-8') as f:
    f.writelines(formatted_data)

print(f"Formatted dataset saved to {output_file}")


Formatted dataset saved to formatted_dataset.txt


In [ ]:
# Read the content from the formatted dataset file
with open(output_file, 'r', encoding='utf-8') as f:
    data = f.read().split("\n\n")  # Split the content into separate records

# Tokenize each sequence individually
tokenized_data = []
for entry in data:
    if entry.strip():  # Ensure that we are not processing empty lines
        # Tokenize each record as one sequence
        tokenized_entry = tokenizer(entry, return_tensors='pt', padding='longest', truncation=True)
        tokenized_data.append(tokenized_entry)

# Print the tokenized data for verification
print("Tokenized data prepared for training.")

# Display each tokenized sequence for review
for i, tokenized_entry in enumerate(tokenized_data):
    decoded_sequence = tokenizer.decode(tokenized_entry['input_ids'][0], skip_special_tokens=True)
    print(f"Sequence {i + 1}:\n{decoded_sequence}\n")


Tokenized data prepared for training.
Sequence 1:
prompt: Who is the author of '1984'?
label: clear
response: George Orwell is the author of '1984.'

Sequence 2:
prompt: What is the largest planet in our Solar System?
label: clear
response: The largest planet in our Solar System is Jupiter.

Sequence 3:
prompt: How many days are in a leap year?
label: clear
response: There are 366 days in a leap year.

Sequence 4:
prompt: The conversation was full of interesting points.
label: unclear
response: What conversation, and what points were interesting?

Sequence 5:
prompt: What does 'HTML' stand for?
label: clear
response: HTML stands for HyperText Markup Language.

Sequence 6:
prompt: What is the main ingredient in guacamole?
label: clear
response: The main ingredient in guacamole is avocado.

Sequence 7:
prompt: The new project will bring significant changes.
label: unclear
response: What project, and what changes are expected?

Sequence 8:
prompt: The bird flew into the open window.
label

In [ ]:
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")
tokenizer.pad_token = tokenizer.eos_token  # Set the pad token to the EOS token

# Read the content from the formatted dataset file
with open(output_file, 'r', encoding='utf-8') as f:
    data = f.read().split("\n\n")  # Split the content into separate records

# Tokenize each sequence
def tokenize_function(text):
    return tokenizer(
        text,
        max_length=600,  # Adjust as needed
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )

# Tokenize all entries and collect input_ids and attention masks
tokenized_texts = [tokenize_function(entry) for entry in data]

# Create a dictionary of input IDs and attention masks
input_ids = torch.cat([t["input_ids"] for t in tokenized_texts], dim=0)
attention_masks = torch.cat([t["attention_mask"] for t in tokenized_texts], dim=0)

# Create a Hugging Face dataset from the tokenized data
dataset = Dataset.from_dict({
    'input_ids': input_ids.tolist(),
    'attention_mask': attention_masks.tolist()
})

In [ ]:
for i in range(5):  # Display 5 examples (adjust as needed)
    input_ids = dataset[i]['input_ids']
    text = tokenizer.decode(input_ids, skip_special_tokens=True)
    print(f"Example {i + 1}:\n{text}\n" + "-" * 50 + "\n")

Example 1:
prompt: Who is the author of '1984'?
label: clear
response: George Orwell is the author of '1984.'
--------------------------------------------------

Example 2:
prompt: What is the largest planet in our Solar System?
label: clear
response: The largest planet in our Solar System is Jupiter.
--------------------------------------------------

Example 3:
prompt: How many days are in a leap year?
label: clear
response: There are 366 days in a leap year.
--------------------------------------------------

Example 4:
prompt: The conversation was full of interesting points.
label: unclear
response: What conversation, and what points were interesting?
--------------------------------------------------

Example 5:
prompt: What does 'HTML' stand for?
label: clear
response: HTML stands for HyperText Markup Language.
--------------------------------------------------



## training model

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Set up training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=2,  # Batch size per device
    gradient_accumulation_steps=4,
    num_train_epochs=5,
    warmup_steps=100,
    learning_rate=2e-4,
    logging_steps=100,
    output_dir='outputs',
    fp16=True  # Use mixed precision
)

# Trainer setup
trainer = Trainer(
    model=loaded_model,
    train_dataset=dataset,
    args=training_args,
    data_collator=data_collator
)



# Disable cache for training
loaded_model.config.use_cache = False

# Train the model
trainer.train()

# Save the trained model
torch.save(loaded_model.state_dict(), 'gpt_neo_finetuned.pt')

# Verify the device of the model
print(f"Model is on device: {next(loaded_model.parameters()).device}")

# Extract the loss values from the trainer's logs
train_loss = []
for log in trainer.state.log_history:
    if 'loss' in log:
        train_loss.append(log['loss'])

# Plot the loss after training
plt.plot(train_loss)
plt.title('Training Loss over Time')
plt.xlabel('Steps (Every 100 steps)')
plt.ylabel('Loss')
plt.show()


Step,Training Loss
100,0.888900
200,0.498700
300,0.429400


KeyboardInterrupt: 

In [ ]:
import os
loaded_model.save_pretrained(os.path.join('/content/drive/MyDrive/cmm', "final_modelsss3"))
tokenizer.save_pretrained(os.path.join('/content/drive/MyDrive/cmm', "final_modelsss3"))

('/content/drive/MyDrive/cmm/final_modelsss3/tokenizer_config.json',
 '/content/drive/MyDrive/cmm/final_modelsss3/special_tokens_map.json',
 '/content/drive/MyDrive/cmm/final_modelsss3/vocab.json',
 '/content/drive/MyDrive/cmm/final_modelsss3/merges.txt',
 '/content/drive/MyDrive/cmm/final_modelsss3/added_tokens.json',
 '/content/drive/MyDrive/cmm/final_modelsss3/tokenizer.json')

## Testing model:

In [ ]:
!ls /content/drive
!rm -rf /content/drive


MyDrive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
# Example prompt for testing
prompt = "prompt: How many days are in a leap year?\nlabel: "
# Tokenize the prompt
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# Generate the output
loaded_model.eval()
with torch.no_grad():
    outputs = loaded_model.generate(
        **inputs,
        max_length=100,
        num_return_sequences=1,
        pad_token_id=tokenizer.pad_token_id
    )

# Decode the output
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Stop processing at the second occurrence of "\nprompt:"
split_response = response.split("\nprompt:", 2)
if len(split_response) > 1:
    final_response = split_response[0] + "\nprompt:" + split_response[1]
else:
    final_response = response

# Print the prompt and the model's response
print("Prompt:")
print(prompt)
print("\nModel Response:")
print(final_response)


Prompt:
prompt: How many days are in a leap year?
label: 

Model Response:
prompt: How many days are in a leap year?
label:  clear
response: There are 366 days in a leap year.
response: What is a leap year?
response: A leap year is a year that has 366 days.
response: What is a leap day?
Response: A leap day is a day that has 366 hours and 21 minutes.
Response: What is a leap hour?
Response: A leap hour is a hour that has 366 minutes


In [9]:
response

"label: clear\nprompt: What is the capital of France?\nresponse: The capital of France is Paris.\nclear: What is the capital of France?\nresponse: The capital of France is Paris.\nclear: Which year was France founded?\nResponse: France was founded in 1791.\nclear: Which year was France founded?\nResponse: 1791 was the year of France's founding.\nclear: Which year was France founded?\nResponse: 1791 was"

## Self-refinement

In [33]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load the GPT-Neo 1.3B model and tokenizer
model_name = "EleutherAI/gpt-neo-1.3B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = loaded_model
model.eval()  # Set to evaluation mode

# Load a pre-trained NLI model for evaluating response coherence
nli_model_name = "facebook/bart-large-mnli"
nli_model = AutoModelForSequenceClassification.from_pretrained(nli_model_name)
nli_tokenizer = AutoTokenizer.from_pretrained(nli_model_name)
nli_model.eval()

# Function for generating initial response
def generate_initial_response(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs,
        max_length=150,
        num_return_sequences=1,
        do_sample=True,
        temperature=0.7
    )
    initial_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return initial_response

# Function for generating a self-review
def generate_self_review(response):
    review_prompt = f"Review the following response and suggest improvements: {response}"
    review_inputs = tokenizer(review_prompt, return_tensors="pt").to(device)
    review_outputs = model.generate(
        **review_inputs,
        max_length=200,
        num_return_sequences=1,
        do_sample=True,
        temperature=0.7
    )
    review = tokenizer.decode(review_outputs[0], skip_special_tokens=True)
    return review

# Function for evaluating with NLI
def evaluate_with_nli(prompt, response):
    nli_input = nli_tokenizer.encode(prompt, response, return_tensors="pt", padding="max_length", truncation=True, max_length=512)
    with torch.no_grad():
        nli_outputs = nli_model(nli_input)
        nli_logits = nli_outputs.logits
        nli_predictions = torch.argmax(nli_logits, dim=-1)
    return nli_predictions.item()

# Function to refine the response based on self-review and NLI
def refine_response(prompt, initial_response, review):
    refined_prompt = f"Improve the following response based on the feedback: {initial_response} Feedback: {review}"
    refined_inputs = tokenizer(refined_prompt, return_tensors="pt").to(device)
    refined_outputs = model.generate(
        **refined_inputs,
        max_length=600,
        num_return_sequences=1,
        do_sample=True,
        temperature=0.7
    )
    refined_response = tokenizer.decode(refined_outputs[0], skip_special_tokens=True)
    return refined_response

# Main function to perform self-refinement with NLI
def generate_response_with_self_refinement(prompt):
    initial_response = generate_initial_response(prompt)
    print(f"Initial Response:\n{initial_response}\n")

    # Generate a self-review
    self_review = generate_self_review(initial_response)
    print(f"Self-Review:\n{self_review}\n")

    # Evaluate initial response with NLI
    nli_score = evaluate_with_nli(prompt, initial_response)
    print(f"NLI Score: {nli_score}\n")

    # Append an NLI message to the feedback if the score is below the threshold
    if nli_score < 1:  # Adjust the threshold as needed
        additional_feedback = "The response is logically not correct. Consider revising the content for coherence."
        self_review += f" {additional_feedback}"

    # Print the updated self-review
    print(f"Updated Self-Review:\n{self_review}\n")

    # Refine the response based on self-review
    refined_response = refine_response(prompt, initial_response, self_review)
    print(f"Refined Response:\n{refined_response}")

# Example usage
prompt = "The novel was adapted into a movie with great success."
generate_response_with_self_refinement(prompt)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Initial Response:
The novel was adapted into a movie with great success. Why is it so successful?

The novel was adapted into a movie with great success. Why is it so successful?

Because it was both a character study and a story about human relationships.

How do you think that movie made you want to read it again?

The movie made me want to read it again because I was enjoying the character study and the story itself.

What kind of character study was it?

It was about a man who was a doctor, but he was also a writer.

What made the doctor a character in the movie?

One of the things that made the doctor a character in the movie was that he

Self-Review:
Review the following response and suggest improvements: The novel was adapted into a movie with great success. Why is it so successful?

The novel was adapted into a movie with great success. Why is it so successful?

Because it was both a character study and a story about human relationships.

How do you think that movie made you wa

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


NLI Score: 1

Updated Self-Review:
Review the following response and suggest improvements: The novel was adapted into a movie with great success. Why is it so successful?

The novel was adapted into a movie with great success. Why is it so successful?

Because it was both a character study and a story about human relationships.

How do you think that movie made you want to read it again?

The movie made me want to read it again because I was enjoying the character study and the story itself.

What kind of character study was it?

It was about a man who was a doctor, but he was also a writer.

What made the doctor a character in the movie?

One of the things that made the doctor a character in the movie was that he had a unique way of communicating with his patients.

What kind of story was it about?

It was a science fiction story about a man who was a doctor, but he was also a writer

Refined Response:
Improve the following response based on the feedback: The novel was adapted into a 